##### Bash_Note.ipynb

- **コンソールで打ち込む場合と、シェルファイルから実行する場合では、挙動が異なる場合がある。**
- **「quote>」という文字が出てきたとき、、、、、**<br>
「'」とか「`」とかクォート系が1個紛れ込むと、ターミナルは入力が続いてると判断するので、<br>
コマンドを打ち切らずに入力を継続させるようになっています。<br>
慌てず騒がずCtrl+cで入力を強制終了しましょう。

- pauseみたいなことをしたいとき。

In [1]:
read -p "Press any key..."

SyntaxError: invalid syntax (<ipython-input-1-2e2dbe38b554>, line 1)

- 算術演算や比較演算したいとき。

In [ ]:

/* ALTER TABLE <テーブル名> DROP PRIMARY KEY; */
/* ALTER TABLE <テーブル名> PARTITION BY ROUNDROBIN PARTITIONS <パーティション数>; */
ALTER TABLE WARDROBE_INFO_TMP1 DROP PRIMARY KEY;
ALTER TABLE WARDROBE_INFO_TMP1 PARTITION BY ROUNDROBIN PARTITIONS 8;


- 改行コードのエスケープ
- 参考：http://osksn2.hep.sci.osaka-u.ac.jp/~taku/osx/crlf.html（nkfかtrコマンドを使って、改行コードを変換できる。）

In [ ]:
/* クリティカルマージとは、デルタマージの大きさが特定の閾値を超えると、HANAが自動的に行うマージのこと。 */
/* スマートマージ */
MERGE DELTA OF <テーブル名> WITH PARAMETERS (‘SMART_MERGE’ = ‘ON');
/* ハードマージ */
MERGE DELTA OF <テーブル名> ;
/* フォースマージは以下のどちらか。 */
MERGE DELTA OF <テーブル名> FORCE REBUILD;
MERGE DELTA OF <テーブル名> WITH PARAMETERS (‘FORCED_MERGE’ = ‘ON');

- シングルクォートのエスケープ

In [ ]:
SELECT winner, subject FROM nobel WHERE yr=1984
 ORDER BY subject IN (
  'Physics', ‘Chemistry'
) ASC, subject ASC, winner ASC;

#### 配列
- 配列を宣言する

In [ ]:
SELECT name, continent FROM world WHERE continent IN (
  SELECT continent FROM world WHERE name IN ('Argentina', 'Australia’)
)
ORDER BY name ASC;

- 配列の長さ

In [ ]:
SELECT name, population FROM world WHERE population BETWEEN (
  SELECT population FROM world WHERE name = ‘Poland’
) 
AND (
  SELECT population FROM world WHERE name = ‘Canada’
);

- 配列のデータを参照する

In [ ]:
# データの先頭要素を参照する
${array[0]} # シェルファイルで打つ場合
${array[1]} # コンソールで打つ場合
# データの末尾要素を参照する
${array[$((${#array[@]}-1))]} # シェルファイルで打つ場合
${array[$((${#array[@]}))]} # コンソールで打つ場合

- 配列にデータを追加する

In [ ]:
# 先頭に追加
array=(3 "${array[@]}")        # array は (3 “a” "b" "c")
# 末尾に追加
array=("${array[@]}" 4)        # array は (3 "a" "b" "c" 4)
array+=( 5 )                   # array は (3 "a" "b" "c" 4 5)

- 配列からデータを取り出す

In [ ]:
# データの先頭要素を取り出す（破壊的操作）
array=("${array[@]:1}")                   # array は ("a" "b" "c" 4 5)
# データの末尾要素取り出す（破壊的操作）
declare -i num=${#array[@]}-1
array=("${array[@]:0:$num}")              # array は ("a" "b" "c" 4)
# 一行でも書けます
array=(${array[@]:0:((${#array[@]}-1))})  # array は ("a" "b" "c")

- 配列からデータを削除する

In [ ]:
# ある要素を削除
unset array[1]
echo "${array[@]}"             # arrayは("a" "c")

#### ECHO
- echo -n（Linuxのみで利用可、echoに改行を含めない）

In [ ]:
echo -n "改行なし"

#### IF条件分岐
- ifの使用例

In [ ]:
if [ $size -eq 0 ]; then
  # ファイルサイズ0なら、ゴミなので安心して削除
  rm /tmp/hoge.txt
elif [ $size -eq 100 ]; then
  # ファイルサイズ100バイトなら、正常なのでここでは何もせず次へ進む。
  :
else
  # ファイルサイズがこれ以外なら、おかしいのでエラー終了
  echo 'Invalid file!' 1>&2
  exit 1
fi

#### CASEで場合分け
- キー入力によって動作を変える（read）
- 参考：https://www.atmarkit.co.jp/ait/articles/1811/28/news003.html

In [ ]:
#! /bin/bash
read str       # 標準入力（キーボード）から1行受け取って変数strにセット
case "$str" in # 変数strの内容で分岐
    [Yy]|[Yy][Ee][Ss])
        echo "YES"
        echo "OK"
        ;;
    [Nn]|[Nn][Oo])
        echo "NO"
        ;;
    *)
        echo "undefined";;
esac

#### FOR繰り返し
- forのフォーマット

In [ ]:
for 繰り返し条件; do
    # ...
done


- 初期値・ループ条件・ループ時の処理で指定する

In [ ]:
max=10
for ((i=0; i < $max; i++)); do
    echo $i
done

- 値を直接指定する

In [ ]:
for i in 0 1 3 5 8 9; do
    echo $i
done

- 値の部分には変数を指定することも可能。

In [ ]:
foo="0"
bar="1 3 5 8 9"
for i in $foo $bar; do
    echo $i
done

- 値を範囲で指定する

In [ ]:
for i in {0..9}; do
    echo $i
done

- inの後方でバッククォートを使ってseqコマンドを実行しても同様のことが実現できる。

In [ ]:
for i in `seq 0 9`; do
    echo $i
done

- バッククォートの部分は$()でも同様のことができる。

In [ ]:
for i in $(seq 0 9); do
    echo $i
done

- 値を配列変数で指定する

array=(0 1 2 3 4 5 6 7 8 9)
for i in ${array[@]}; do
    echo $i
done

- 値をファイルで指定する<br>
number.txt 

0
1
2
3
4
5
6
7
8
9

for i in $(cat number.txt); do
    echo $i 
done

- 値にコマンドライン引数を利用する

for i in "$@"; do
    echo $i
done

#### DECLARE
- declare 変数名（変数を定義する）

declare a

- declare 変数名=値（変数を定義する際に、値もセットする）

declare a=‘abc'

- declare -p 変数名（変数の定義内容を表示する）

declare -p a

- declare -p（現在定義されている変数の名前と定義内容を一覧表示する）

declare -p

- declare -F（現在定義されている関数の名前を一覧表示する）

declare -F | head

- declare -f 関数名（関数の内容を表示する）

declare -f quote

- declare -f（現在定義されている関数の名前と内容を一覧表示する）

declare -f

#### TRコマンドで文字列を置換や削除
- tr 文字セット1 [文字セット2]（文字セット1を文字セット2に置換する。）
- 参考：https://www.atmarkit.co.jp/ait/articles/1610/03/news017.html

In [ ]:
# 改行を消せる。
tr '\012' ' '

- tr -d 文字セット1（文字セット1を削除する。）

In [ ]:
tr -d '\012'

#### Linuxマシンの情報を取得する
- （CPUコア数）＝（物理CPUの数）×（CPUごとのコアの数）
- 物理CPUの数

grep physical.id /proc/cpuinfo | sort -u | wc -l

- CPUごとのコアの数

grep cpu.cores /proc/cpuinfo | sort -u

- 論理プロセッサーの数

grep processor /proc/cpuinfo | wc -l

#### プロセスやCPU状況を監視
- 参考：https://qiita.com/aosho235/items/c4d6995743dd1dac16e1
- psでプロセスの様子をチェックする<br>
（-eは全てのプロセスを表示、-fはプロセスの親子関係を表示）

ps -ef | grep <プロセス名 or プロセスID>

- topでリアルタイム監視（CPUやメモリを食っているプロセスを特定したい場合）

top